In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
pip install -q -U "tensorflow-text==2.8.*" 

In [3]:
pip install -q tf-models-official==2.7.0

In [ ]:
import os
import shutil
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('/content/waseem_v1.csv', header = None)
df.drop(0, axis=1, inplace = True)

In [7]:
df = df.sample(frac = 1)
#df.reset_index(inplace = True)
#df.drop('level_0', inplace = True, axis=1)
df.reset_index(inplace = True)
df

,index,1,2
0,3239,Lynn and Tony being so strategic and gave a 3 ...,0.0
1,618,RT QuestionsForMen This one s good and top...,1.0
2,7746,hahaha the only anime wall scroll I own is ...,0.0
3,7167,not necessarily,0.0
4,9280,Yes all men disappoint eventually,0.0
...,...,...,...
9948,9948,RT just so I m clear you have dogs named ...,0.0
9949,5878,Donating blood tomorrow Pretty sure that I bl...,0.0
9950,3333,Yes the dish is refined nothing says refined...,0.0
9951,3199,I agree with a 4 Will and Steve mkr,0.0


In [8]:
df.drop('index', inplace = True, axis=1)

In [9]:
df

,1,2
0,Lynn and Tony being so strategic and gave a 3 ...,0.0
1,RT QuestionsForMen This one s good and top...,1.0
2,hahaha the only anime wall scroll I own is ...,0.0
3,not necessarily,0.0
4,Yes all men disappoint eventually,0.0
...,...,...
9948,RT just so I m clear you have dogs named ...,0.0
9949,Donating blood tomorrow Pretty sure that I bl...,0.0
9950,Yes the dish is refined nothing says refined...,0.0
9951,I agree with a 4 Will and Steve mkr,0.0


In [10]:
train_df = df.iloc[0:7464, :]
test_df = df.iloc[7465:8708, :]
val_df = df.iloc[8709:, :]
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7464 entries, 0 to 7463
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       7462 non-null   object 
 1   2       7462 non-null   float64
dtypes: float64(1), object(1)
memory usage: 116.8+ KB


In [ ]:
pip install transformers

In [12]:
import transformers 
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification,TFBertForSequenceClassification
from transformers import AutoTokenizer
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

In [13]:
x_train, y_train = list(train_df[1].astype(str)), list(train_df[2])
type(x_train)

list

In [ ]:
train_encodings = tokenizer(x_train, padding=True, truncation=True)
train_encodings

In [36]:
train_ds = tf.data.Dataset.from_tensor_slices((
      dict(train_encodings),
      y_train
  ))
train_ds = train_ds.batch(2)
train_ds

<BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 83), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(None, 83), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 83), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [16]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2 )


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-5,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=tf.metrics.SparseCategoricalAccuracy()
              )

In [38]:
model.fit(x=train_ds, epochs=1)

1000/3732 [=======>......................] - ETA: 1:39:14 - loss: 0.4999 - sparse_categorical_accuracy: 0.8000

KeyboardInterrupt: ignored